# Conceptual Chain - Analyze a news article and decide business impact

Input Article 

          ↓
   
Prompt 1 → Summarize

          ↓

Prompt 2 → Analyze risks & opportunities

          ↓

Prompt 3 → Make a decision


In [5]:
def log_step(step_name: str, output: str):
    print(f"\n{'='*20} {step_name.upper()} {'='*20}")
    print(output)

In [6]:
import os
import time
from openai import OpenAI
from dotenv import load_dotenv

# ------------------ Setup ------------------
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MODEL_NAME = "gpt-4.1-mini"   
TEMPERATURE = 0.2
MAX_RETRIES = 3


# ------------------ Core LLM Call ------------------
def call_llm(prompt: str, system_role: str = "You are a helpful assistant.") -> str:
    """
    Centralized LLM call with retries and deterministic behavior.
    """
    for attempt in range(MAX_RETRIES):
        try:
            response = client.responses.create(
                model=MODEL_NAME,
                input=[
                    {"role": "system", "content": system_role},
                    {"role": "user", "content": prompt}
                ],
                temperature=TEMPERATURE
            )

            return response.output_text.strip()

        except Exception as e:
            if attempt == MAX_RETRIES - 1:
                raise e
            time.sleep(2 ** attempt)  # exponential backoff


# ------------------ Step 1: Summarization ------------------
def summarize_article(article: str) -> str:
    prompt = f"""
    Summarize the following article in exactly 5 bullet points.
    Focus only on key factual information.

    Article:
    {article}
    """
    return call_llm(prompt, system_role="You are a professional business analyst.")


# ------------------ Step 2: Analysis ------------------
def analyze_summary(summary: str) -> str:
    prompt = f"""
    Analyze the summary below.
    Identify:
    - Key risks
    - Key opportunities

    Summary:
    {summary}
    """
    return call_llm(prompt, system_role="You are a senior strategy consultant.")


# ------------------ Step 3: Decision ------------------
def make_decision(analysis: str) -> str:
    prompt = f"""
    Based on the analysis below,
    recommend one action:
    - Proceed
    - Hold
    - Reject

    Provide a short justification.

    Analysis:
    {analysis}
    """
    return call_llm(prompt, system_role="You are an executive decision-maker.")


# ------------------ Orchestrator ------------------
def prompt_chain(article: str) -> str:
    summary = summarize_article(article)
    log_step("Summarization", summary)
    analysis = analyze_summary(summary)
    log_step("Analysis", analysis)
    decision = make_decision(analysis)
    log_step("Decision", decision)
    return decision


# ------------------ Execution ------------------
if __name__ == "__main__":
    article_text = (
        "Company X reported a 20% revenue growth in Q4, "
        "driven by strong demand in emerging markets, "
        "but operational costs increased significantly due to logistics and hiring."
    )

    final_decision = prompt_chain(article_text)
    print("\nFINAL DECISION:\n", final_decision)



==================== SUMMARIZATION ====================
- Company X experienced a 20% revenue growth in Q4.  
- The revenue increase was driven by strong demand in emerging markets.  
- Operational costs rose significantly during the quarter.  
- Increased costs were primarily due to logistics expenses.  
- Hiring activities also contributed to the higher operational costs.

==================== ANALYSIS ====================
Certainly. Here's the analysis of the summary provided:

**Key Risks:**
1. **Rising Operational Costs:** The significant increase in operational costs, especially due to logistics expenses, could erode profit margins if not managed effectively.
2. **Sustainability of Revenue Growth:** The 20% revenue growth is driven mainly by emerging markets, which can be volatile and subject to geopolitical, economic, or regulatory risks.
3. **Hiring-Related Expenses:** Increased hiring activities may lead to higher fixed costs and potential challenges in workforce integration 